### Clifford QSVT 

In this notebook, I apply the quantum singular value transform to three different clifford circuits.

In [2]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
from timeit import timeit

# constructing stabilizer (clifford) circuits
dev1 = qml.device("default.clifford", wires=2)
dev2 = qml.device("default.clifford", wires=2)
dev3 = qml.device("default.clifford", wires=4)

@qml.qnode(dev1)
# clifford 1 has 2 quibits with a depth of 3
def clifford1(return_state=True):
    qml.X(wires=[0])
    qml.CNOT(wires=[0, 1])
    qml.S(wires=[0])
    qml.Hadamard(wires=[1])
    return qml.probs(), ([qml.state()] if return_state else [])

@qml.qnode(dev2)
# clifford 1 has 2 quibits with a depth of 5
def clifford2(return_state=True):
    qml.Hadamard(wires=[0])
    qml.X(wires=[0])
    qml.CY(wires=[0, 1])
    qml.Z(wires=[1])
    qml.S(wires=[1])
    return qml.probs(), ([qml.state()] if return_state else [])

@qml.qnode(dev3)
# clifford 1 has 4 quibits with a depth of 5
def clifford3(return_state=True):
    qml.S(wires=[0])
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires=[1])
    qml.CY(wires=[1, 2])
    qml.adjoint(qml.S(wires=[2]))
    qml.CZ(wires=[2, 3])
    return qml.probs(), ([qml.state()] if return_state else [])

In [3]:
# converting circuits to matrix form

cliff1_matrix = qml.matrix(clifford1)().round(3) 
cliff2_matrix = qml.matrix(clifford2)().round(3)  
cliff3_matrix = qml.matrix(clifford3)().round(3)  

print(cliff1_matrix)

[[ 0.   +0.j     0.   +0.j     0.707+0.j     0.707+0.j   ]
 [ 0.   +0.j     0.   +0.j     0.707+0.j    -0.707+0.j   ]
 [ 0.   +0.707j  0.   +0.707j  0.   +0.j     0.   +0.j   ]
 [ 0.   -0.707j  0.   +0.707j  0.   +0.j     0.   +0.j   ]]


In [4]:
# # block encoding the circuits
# block1 = qml.BlockEncode(cliff1_matrix, wires=range(3))
# block2 = qml.BlockEncode(cliff2_matrix, wires = range(3))
# block3 = qml.BlockEncode(cliff3_matrix, wires = range(5))

In [22]:
# we approximate the identity polynomial
id_poly = np.array([0,0.5, 0, 0.5])

qsvt_cliff1 = qml.matrix(qml.qsvt, wire_order=list(range(3)))(
    cliff1_matrix, id_poly, encoding_wires=list(range(3)), block_encoding="embedding"
) # block encoded in 3-qubit system - 1 ancilla required for embedding

print(qsvt_cliff1)
qsvt_cliff1.shape
# we use fidelity to compare the transformed state with the original state
# fidelity1 = qml.math.fidelity(qsvt_cliff1, cliff1_matrix)

[[ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   7.06893243e-01-2.81453249e-04j  7.06893243e-01-2.81453249e-04j
   1.14876704e-02-2.17200599e-02j -2.48667509e-19-2.64198572e-19j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   7.06893243e-01-2.81453249e-04j -7.06893243e-01+2.81453249e-04j
  -5.92431893e-19+1.36446569e-18j  1.14876704e-02-2.17200599e-02j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 2.81453249e-04+7.06893243e-01j  2.81453249e-04+7.06893243e-01j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   1.14876704e-02-2.17200599e-02j -8.53127734e-19-1.19426416e-18j]
 [-2.81453249e-04-7.06893243e-01j  2.81453249e-04+7.06893243e-01j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   6.68

(8, 8)

In [ ]:
U_cliff2 = qml.matrix(qml.qsvt, wire_order=list(range(3)))(
    cliff2_matrix, id_poly, encoding_wires=list(range(3)), block_encoding="embedding"
) # block encoded in 3-qubit system - 1 ancilla required for embedding

qsvt_cliff2 = np.real(np.diag(U_cliff1))[:16] 

In [ ]:
U_cliff3 = qml.matrix(qml.qsvt, wire_order=list(range(5)))(
    cliff2_matrix, id_poly, encoding_wires=list(range(5)), block_encoding="embedding"
) # block encoded in 3-qubit system - 1 ancilla required for embedding

qsvt_cliff3 = np.real(np.diag(U_cliff1))[:16] 